In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install autogluon
!pip install mxnet~=1.9

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tqdm import tqdm

import holidays
from autogluon.tabular import TabularDataset, TabularPredictor

import warnings
warnings.filterwarnings('ignore')

In [ ]:
path = '/content/drive/MyDrive/DACON/jeju_price/'

In [ ]:
train = pd.read_csv(path + 'train.csv')
international_trade = pd.read_csv(path + 'international_trade.csv')
test = pd.read_csv(path + 'test.csv')

train['item_id'] = train.ID.str[0:6]
train = train[(train["item_id"] != "BC_B_S") & (train["item_id"] != "BC_C_S") & (train["item_id"] != "CB_A_S") & (train["item_id"] != "CR_D_S") & (train["item_id"] != "CR_E_S") & (train["item_id"] != "RD_C_S")]
train = train.reset_index(drop = True)
train.drop(columns=['item_id'], inplace=True)

test['item_id'] = test.ID.str[0:6]
test = test[(test["item_id"] != "BC_B_S") & (test["item_id"] != "BC_C_S") & (test["item_id"] != "CB_A_S") & (test["item_id"] != "CR_D_S") & (test["item_id"] != "CR_E_S") & (test["item_id"] != "RD_C_S")]
test = test.reset_index(drop = True)
test.drop(columns=['item_id'], inplace=True)

In [ ]:
def group_season(df):
    df.loc[(df['month'] == 3) | (df['month'] == 4) | (df['month'] == 5), 'season'] = '봄'
    df.loc[(df['month'] == 6) | (df['month'] == 7) | (df['month'] == 8), 'season'] = '여름'
    df.loc[(df['month'] == 9) | (df['month'] == 10) | (df['month'] == 11), 'season'] = '가을'
    df.loc[(df['month'] == 12) | (df['month'] == 1) | (df['month'] == 2), 'season'] = '겨울'
    return df['season']

def holiday(df):
    kr_holidays = holidays.KR()
    df['holiday'] = df.timestamp.apply(lambda x: 'holiday' if x in kr_holidays else 'non-holiday')
    return df['holiday']

def cyclical_feature(df, time=12):
    df['sin_time'] = np.sin(2*np.pi*df.month/time)
    df['cos_time'] = np.cos(2*np.pi*df.month/time)

def post_preprocessing(test, submission):
    idx_list = test[(test['Weekday'] == 6)].index
    submission.loc[idx_list, 'answer'] = 0 # Weekday == 6 (일요일)이면 가격 0원
    submission['answer'] = submission['answer'].apply(lambda x: max(0, x)) # 가격에 음수가 있다면 가격 0원으로 변경
    return submission

def determine_harvest_weight(item, month):
    harvest_times = {
    'TG': {'main': [(10, 1)]},  # 감귤: 10월부터 이듬해 1월까지
    'BC': {'main': [(4, 6), (9, 11)]},  # 브로콜리: 4월-6월, 9월-11월
    'RD': {'main': [(5, 6), (11, 12)]},  # 무: 5월, 11월
    'CR': {'main': [(7, 8), (10, 11)]},  # 당근: 7월-8월, 10월-12월
    'CB': {'main': [(6, 6), (11, 11)]}  # 양배추: 6월, 11월
}
    main_harvest = harvest_times[item]['main']
    for start, end in main_harvest:
        if start <= month <= end:
            return 1
    return 0

In [ ]:
class DataPreprocessing:
    def __init__(self, train, test):
        self.train = train
        self.test = test

    @staticmethod
    def label_encode(train, test):
        categorical_col = ['item', 'corporation', 'location', 'season', 'holiday', 'total_item_value',
                           'item_month_Weekday', 'item_corp_Weekday', 'item_location_Weekday', 'item_year_season', 'item_weight']

        for i in categorical_col:
            le = LabelEncoder()
            train[i] = le.fit_transform(train[i])
            test[i] = le.transform(test[i])

        return train, test

    @staticmethod
    def remove_outliers(train):
        print('Remove outliers')
        train.loc[(train['Weekday'] == 6) & (train['price(원/kg)'] >= 0), 'price(원/kg)'] = 0
        return train

    @staticmethod
    def preprocessing(data):
        print('Preprocessing Start')
        # time feature
        data['year'] = data['timestamp'].apply(lambda x: int(x[0:4]))
        data['month'] = data['timestamp'].apply(lambda x: int(x[5:7]))
        data['Weekday'] = pd.to_datetime(data['timestamp']).dt.weekday
        data['is_weekend'] = data['Weekday'].apply(lambda x: 1 if x >= 6 else 0)
        data['year'] = data['year'] - 2019
        data['season'] = group_season(data)
        data['holiday'] = holiday(data)
        cyclical_feature(data)

        # item feature
        data['total_item_value'] = data['item']+data['corporation']+data['location']
        data['item_month_Weekday'] = data['item'].astype(str) + "_" + data['month'].astype(str) + data['Weekday'].astype(str)
        data['item_corp_Weekday'] = data['item'].astype(str) + "_" + data['corporation'].astype(str) + data['Weekday'].astype(str)
        data['item_location_Weekday'] = data['item'].astype(str) + "_" + data['location'].astype(str) + data['Weekday'].astype(str)
        data['item_year_season'] = data['item'].astype(str) + "_" + data['year'].astype(str) + "_" + data['season'].astype(str)


        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data['harvest_weight'] = data.apply(lambda row: determine_harvest_weight(row['item'], row['timestamp'].month), axis=1)
        # data['timestamp'] = data['timestamp'].view('int64') * 1e9

        data['item_weight'] = data['item'].astype(str) + "_" + data['harvest_weight'].astype(str)
        return data

    def fit(self):
        self.train = self.preprocessing(self.train)
        self.test = self.preprocessing(self.test)

        self.train = self.remove_outliers(self.train)

        x_train = self.train.drop(columns=['ID', 'supply(kg)', 'price(원/kg)'])
        y_train = self.train['price(원/kg)']
        x_test = self.test.drop(columns=['ID'])

        # x_train, x_test = self.label_encode(x_train, x_test)

        return x_train, y_train, x_test

In [ ]:
preprocessing = DataPreprocessing(train, test)
x, y, test = preprocessing.fit()
train_set = pd.concat([x, y], axis=1)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=1103)
train_autogluon = pd.concat([x, y], axis=1)


train_autogluon['week_of_year'] = train_autogluon['timestamp'].dt.isocalendar().week
test['week_of_year'] = test['timestamp'].dt.isocalendar().week

train_autogluon['week_item'] = train_autogluon['week_of_year'].astype(str) + '_' + train_autogluon['item'].astype(str)
test['week_item'] = test['week_of_year'].astype(str) + '_' + test['item'].astype(str)
filtered_data = train_autogluon[train_autogluon['price(원/kg)'] > 0]
mean_encoded = filtered_data.groupby('week_item')['price(원/kg)'].mean().reset_index()
mean_encoded.rename(columns={'price(원/kg)': 'mean_encoded_price'}, inplace=True)
train_autogluon = pd.merge(train_autogluon, mean_encoded, on='week_item', how='left')
test = pd.merge(test, mean_encoded, on='week_item', how='left')

train_autogluon['week_item_location'] = train_autogluon['week_of_year'].astype(str) + '_' + train_autogluon['item'].astype(str) + '_' + train_autogluon['location'].astype(str)
test['week_item_location'] = test['week_of_year'].astype(str) + '_' + test['item'].astype(str) + '_' + test['location'].astype(str)
filtered_data = train_autogluon[train_autogluon['price(원/kg)'] > 0]
mean_encoded_mwic = filtered_data.groupby('week_item_location')['price(원/kg)'].mean().reset_index()
mean_encoded_mwic.rename(columns={'price(원/kg)': 'mean_encoded_price_mwil'}, inplace=True)
train_autogluon = pd.merge(train_autogluon, mean_encoded_mwic, on='week_item_location', how='left')
test = pd.merge(test, mean_encoded_mwic, on='week_item_location', how='left')

train_autogluon['week_item_corporation'] = train_autogluon['week_of_year'].astype(str) + '_' + train_autogluon['item'].astype(str) + '_' + train_autogluon['corporation'].astype(str)
test['week_item_corporation'] = test['week_of_year'].astype(str) + '_' + test['item'].astype(str) + '_' + test['corporation'].astype(str)
filtered_data_mwic = train_autogluon[train_autogluon['price(원/kg)'] > 0]
mean_encoded_mwic = filtered_data_mwic.groupby('week_item_corporation')['price(원/kg)'].mean().reset_index()
mean_encoded_mwic.rename(columns={'price(원/kg)': 'mean_encoded_price_mwic'}, inplace=True)
train_autogluon = pd.merge(train_autogluon, mean_encoded_mwic, on='week_item_corporation', how='left')
test = pd.merge(test, mean_encoded_mwic, on='week_item_corporation', how='left')

train_autogluon['week_item_corporation_location'] = train_autogluon['week_of_year'].astype(str) + '_' + train_autogluon['item'].astype(str) + '_' + train_autogluon['corporation'].astype(str) + '_' + train_autogluon['location'].astype(str)
test['week_item_corporation_location'] = test['week_of_year'].astype(str) + '_' + test['item'].astype(str) + '_' + test['corporation'].astype(str) + '_' + test['location'].astype(str)
filtered_data_mwicl = train_autogluon[train_autogluon['price(원/kg)'] > 0]
mean_encoded_mwicl = filtered_data_mwicl.groupby('week_item_corporation_location')['price(원/kg)'].mean().reset_index()
mean_encoded_mwicl.rename(columns={'price(원/kg)': 'mean_encoded_price_mwicl'}, inplace=True)
train_autogluon = pd.merge(train_autogluon, mean_encoded_mwicl, on='week_item_corporation_location', how='left')
test = pd.merge(test, mean_encoded_mwicl, on='week_item_corporation_location', how='left')



# train_autogluon['original_index'] = train_autogluon.index
# test['original_index'] = test.index

# train_autogluon['item_id'] = train_autogluon['item'].astype(str) + '_' + train_autogluon['corporation'].astype(str) + '_' + train_autogluon['location'].astype(str)
# test['item_id'] = test['item'].astype(str) + '_' + test['corporation'].astype(str) + '_' + test['location'].astype(str)
# weekly_avg = train_autogluon.groupby(['item_id', 'week_of_year'])['mean_encoded_price_mwicl'].mean().reset_index()
# weekly_avg['weekly_difference'] = weekly_avg.groupby('item_id')['mean_encoded_price_mwicl'].diff()
# weekly_avg = weekly_avg[['item_id', 'week_of_year', 'weekly_difference']]
# train_autogluon = pd.merge(train_autogluon, weekly_avg, on=['item_id', 'week_of_year'], how='outer')
# test = pd.merge(test, weekly_avg, on=['item_id', 'week_of_year'], how='outer')
# train_autogluon = train_autogluon.sort_values(by='original_index')
# test = test.sort_values(by='original_index')


# train_autogluon['item_id'] = train_autogluon['item'].astype(str) + '_' + train_autogluon['corporation'].astype(str)
# test['item_id'] = test['item'].astype(str) + '_' + test['corporation'].astype(str)
# weekly_avg = train_autogluon.groupby(['item_id', 'week_of_year'])['mean_encoded_price_mwic'].mean().reset_index()
# weekly_avg['weekly_difference_2'] = weekly_avg.groupby('item_id')['mean_encoded_price_mwic'].diff()
# weekly_avg = weekly_avg[['item_id', 'week_of_year', 'weekly_difference_2']]
# train_autogluon = pd.merge(train_autogluon, weekly_avg, on=['item_id', 'week_of_year'], how='outer')
# test = pd.merge(test, weekly_avg, on=['item_id', 'week_of_year'], how='outer')
# train_autogluon = train_autogluon.sort_values(by='original_index')
# test = test.sort_values(by='original_index')

# train_autogluon['item_id'] = train_autogluon['item'].astype(str) + '_' + train_autogluon['location'].astype(str)
# test['item_id'] = test['item'].astype(str) + '_' + test['location'].astype(str)
# weekly_avg = train_autogluon.groupby(['item_id', 'week_of_year'])['mean_encoded_price_mwil'].mean().reset_index()
# weekly_avg['weekly_difference_3'] = weekly_avg.groupby('item_id')['mean_encoded_price_mwil'].diff()
# weekly_avg = weekly_avg[['item_id', 'week_of_year', 'weekly_difference_3']]
# train_autogluon = pd.merge(train_autogluon, weekly_avg, on=['item_id', 'week_of_year'], how='outer')
# test = pd.merge(test, weekly_avg, on=['item_id', 'week_of_year'], how='outer')
# train_autogluon = train_autogluon.sort_values(by='original_index')
# test = test.sort_values(by='original_index')

# train_autogluon['item_id'] = train_autogluon['item'].astype(str)
# test['item_id'] = test['item'].astype(str)
# weekly_avg = train_autogluon.groupby(['item_id', 'week_of_year'])['mean_encoded_price'].mean().reset_index()
# weekly_avg['weekly_difference_4'] = weekly_avg.groupby('item_id')['mean_encoded_price'].diff()
# weekly_avg = weekly_avg[['item_id', 'week_of_year', 'weekly_difference_4']]
# train_autogluon = pd.merge(train_autogluon, weekly_avg, on=['item_id', 'week_of_year'], how='outer')
# test = pd.merge(test, weekly_avg, on=['item_id', 'week_of_year'], how='outer')
# train_autogluon = train_autogluon.sort_values(by='original_index')
# test = test.sort_values(by='original_index')

train_autogluon['timestamp'] = train_autogluon['timestamp'].view('int64') * 1e9
test['timestamp'] = test['timestamp'].view('int64') * 1e9

# train_autogluon.drop(columns=['item_id', 'original_index'], inplace=True)
# test.drop(columns=['item_id', 'original_index'], inplace=True)

# train_autogluon = train_autogluon.fillna(0)
# test = test.fillna(0)

# cat_mean_col = ['total_item_value', 'item_month_Weekday', 'item_corp_Weekday', 'item_location_Weekday', 'item_year_season']
# for cat_col in cat_mean_col:
#     mean_value = pd.pivot_table(train_autogluon, values = 'price(원/kg)', index = [f'{cat_col}'], aggfunc = np.mean).reset_index()
#     tqdm.pandas()
#     train_autogluon[f'{cat_col}_mean'] = train_autogluon.progress_apply(lambda x : mean_value.loc[(mean_value[f'{cat_col}'] == x[f'{cat_col}']),'price(원/kg)'].values[0], axis = 1)
#     tqdm.pandas()
#     test[f'{cat_col}_mean'] = test.progress_apply(lambda x : mean_value.loc[(mean_value[f'{cat_col}'] == x[f'{cat_col}']) ,'price(원/kg)'].values[0], axis = 1)

#     std_value = pd.pivot_table(train_autogluon, values = 'price(원/kg)', index = [f'{cat_col}'], aggfunc = np.std).reset_index()
#     tqdm.pandas()
#     train_autogluon[f'{cat_col}_std'] = train_autogluon.progress_apply(lambda x : std_value.loc[(std_value[f'{cat_col}'] == x[f'{cat_col}']),'price(원/kg)'].values[0], axis = 1)
#     tqdm.pandas()
#     test[f'{cat_col}_std'] = test.progress_apply(lambda x : std_value.loc[(std_value[f'{cat_col}'] == x[f'{cat_col}']) ,'price(원/kg)'].values[0], axis = 1)

Preprocessing Start
Preprocessing Start
Remove outliers


In [ ]:
hyperparameters = {
    'GBM': [
        {'device': 'gpu', 'extra_trees': True, 'ag_args': {'name_suffix': 'XT'}},
        {'device': 'gpu'},
        'GBMLarge'
    ],
    'CAT': {'task_type': 'GPU'},
    'XGB': {'tree_method': 'gpu_hist'}
}

In [ ]:
label = 'price(원/kg)'
predictor = TabularPredictor(label=label, path=path + 'auto_result').fit(train_autogluon,
                            ag_args_fit={'num_gpus': 1},
                            hyperparameters=hyperparameters,
                            time_limit=3600, presets='best_quality')

In [ ]:
model_list = list(predictor.leaderborad()['model'].unique())

In [ ]:
for model in model_list:
    predictions = predictor.predict(test, model=f'{model}')
    submission = pd.read_csv(path + 'sample_submission.csv')

    submission["item_id"] = submission["ID"].str[0:6]
    submission["TIME"] = submission["ID"].str[10:]

    submission = submission[(submission["item_id"] != "BC_B_S") & (submission["item_id"] != "BC_C_S") & (submission["item_id"] != "CB_A_S") & (submission["item_id"] != "CR_D_S") & (submission["item_id"] != "CR_E_S") & (submission["item_id"] != "RD_C_S")]
    submission = submission.reset_index(drop = True)

    submission['answer'] = np.round(predictions)
    sunday_ids_march_2023 = [f"_202303{day:02}" for day in [5, 12, 19, 26]]
    submission.loc[submission['answer'] < 0.0, 'answer'] = 0.0
    for sunday_id in sunday_ids_march_2023:
        submission.loc[submission['ID'].str.contains(sunday_id), 'answer'] = 0
    # submission.to_csv(path + f'auto_mb_sep/{model}_go.csv', index=False)

    sb = pd.read_csv(path + 'autogluon_submission_feature2.csv')
    sb["item_id"] = sb["ID"].str[0:6]
    sb["TIME"] = sb["ID"].str[10:]

    sb['original_order'] = range(len(sb))

    # 인덱스 설정
    submission.set_index(['item_id', 'TIME'], inplace=True)
    sb.set_index(['item_id', 'TIME'], inplace=True)

    # 공통 인덱스 찾기
    common_indices = sb.index.intersection(submission.index)

    # 'original_order' 열 보존하면서 sb 업데이트
    sb_update = sb.loc[common_indices].copy()
    sb_update.update(submission.loc[common_indices])
    sb.loc[common_indices] = sb_update

    # 인덱스 리셋 및 원래 순서로 정렬
    sb.reset_index(inplace=True)
    sb.sort_values(by='original_order', inplace=True)

    # 불필요한 'original_order' 열 제거
    sb.drop(columns=['original_order'], inplace=True)
    sb.drop(['item_id','TIME'],axis=1,inplace=True)
    sb.to_csv(path + f'autogluon_regression_final_{model}.csv', index=False)